# A.8 高级数组输入输出

我在第4章中讲过，np.save 和 np.load 可用于读写磁盘上以二进制格式存储的数组。其实还有一些工具可用于更为复杂的场景。尤其是内存映像（memory map），它使你能处理在内存中放不下的数据集。

## 内存映像文件

内存映像文件是一种将磁盘上的非常大的 `二进制数据文件` 当做内存中的数组进行处理的方式。NumPy实现了一个类似于ndarray的memmap对象，它允许将大文件分成小段进行读写，而不是一次性将整个数组读入内存。另外，memmap也拥有跟普通数组一样的方法，因此，基本上只要是能用于ndarray的算法就也能用于memmap。

要创建一个内存映像，可以使用函数np.memmap并传入一个文件路径、数据类型、形状以及文件模式：

In [1]:
import numpy as np

In [2]:
mmap = np.memmap('mymmap', 
                 dtype='float64', 
                 mode='w+',
                 shape=(10000, 10000)
                )

mmap

memmap([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

对memmap切片将会返回磁盘上的数据的视图：

In [3]:
section = mmap[:5]

如果将数据赋值给这些视图：数据会先被缓存在内存中（就像是Python的文件对象），调用flush即可将其写入磁盘：

In [4]:
section[:] = np.random.randn(5, 10000)

mmap.flush()

mmap

memmap([[ 0.05704397, -1.83505687,  0.89498322, ...,  0.38314487,
          0.4159928 ,  0.603153  ],
        [-1.009654  , -0.7755939 , -1.17773394, ...,  0.32369958,
         -0.39847625, -0.55877856],
        [ 1.04994376, -2.18208884,  1.30411099, ...,  1.19723281,
         -0.64248139, -0.38442695],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

In [5]:
del mmap

只要某个内存映像超出了作用域，它就会被垃圾回收器回收，之前对其所做的任何修改都会被写入磁盘。当打开一个已经存在的内存映像时，仍然需要指明数据类型和形状，因为磁盘上的那个文件只是一块二进制数据而已，没有任何元数据：

In [6]:
mmap = np.memmap('mymmap', dtype='float64', shape=(10000, 10000))

mmap

memmap([[ 0.05704397, -1.83505687,  0.89498322, ...,  0.38314487,
          0.4159928 ,  0.603153  ],
        [-1.009654  , -0.7755939 , -1.17773394, ...,  0.32369958,
         -0.39847625, -0.55877856],
        [ 1.04994376, -2.18208884,  1.30411099, ...,  1.19723281,
         -0.64248139, -0.38442695],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

内存映像可以使用前面介绍的结构化或嵌套dtype。

## HDF5及其他数组存储方式

PyTables和h5py这两个Python项目可以将NumPy的数组数据存储为高效且可压缩的HDF5格式（HDF意思是“层次化数据格式”）。你可以安全地将好几百GB甚至TB的数据存储为HDF5格式。要学习Python使用HDF5，请参考pandas线上文档。